In [1]:
#Test for git aliases (e.g., git-update-plus, git-next-checkin, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [2]:
# Global Setup
# Creating temp local repos to test on

TMP=/tmp/temp-git-test
TMPB=/tmp/temp-git-test-B

git clone https://github.com/LorenzoFerraces/bash-test.git "$TMP"
git clone https://github.com/LorenzoFerraces/bash-test.git "$TMPB"

# Move to second local to push some changes
command cd "$TMPB" 
git pull origin main 
echo "hello second local" > hello.txt
git add hello.txt
git commit -m "first commit on second local"
git push



Cloning into '/tmp/temp-git-test'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 145 (delta 24), reused 8 (delta 0), pack-reused 113
Receiving objects: 100% (145/145), 12.40 KiB | 317.00 KiB/s, done.
Resolving deltas: 100% (49/49), done.
Cloning into '/tmp/temp-git-test-B'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 145 (delta 24), reused 8 (delta 0), pack-reused 113
Receiving objects: 100% (145/145), 12.40 KiB | 317.00 KiB/s, done.
Resolving deltas: 100% (49/49), done.
From https://github.com/LorenzoFerraces/bash-test
 * branch            main       -> FETCH_HEAD
Already up to date.
bash: hello.txt: cannot overwrite existing file
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


In [3]:
# Moving to first local
command cd "$TMP"

In [4]:
# git-branch-alias test
# switch to new branch

git checkout -b newBranch;
actual_branch=$(git branch --show-current)


Switched to a new branch 'newBranch'


In [5]:
# Assert
[ $(echo $actual_branch) == "$(git-branch-alias)" ]; echo $?;

0


In [6]:
#git-checkout-branch test
git-checkout-branch --help


usage: git-checkout-branch [--help | branch]
note: available branches:
    main


In [7]:
git-checkout-branch main
actual_branch=$(git branch --show-current)

issuing: git checkout main --
Switched to branch 'main'
Your branch is up to date with 'origin/main'.


In [8]:
# Assert
[ $(echo $actual_branch) == "main" ]; echo $?;

0


In [9]:
# git-update-plus test
git-update-plus

No changed files so no pre-stash-pop timestamp check
issuing: git stash
issuing: git pull --all
issuing: git stash pop
No stash entries found.
No changed files so no post-stash-pop timestamp check
No local changes to save
Already up to date.



In [10]:
# Assert
[ "$(cat hello.txt)" == "hello second local" ]; echo $?;

0


In [11]:
# Setup
# adding changes to not be able to pull
command cd "$TMPB"
echo "second change" > hello2.txt
git add . ; git commit -m "second commit"; git push 

command "$TMP"
echo "second line" >> hello.txt


[main 4cf7bac] second commit
 1 file changed, 1 insertion(+)
 create mode 100644 hello2.txt
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 6 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 319 bytes | 319.00 KiB/s, done.
Total 3 (delta 0), reused 2 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   4732e90..4cf7bac  main -> main
bash: /tmp/temp-git-test: Is a directory


In [12]:
# Should fail
git-update-plus

Error: Use 'git-update-force' to update with changed files (n.b., potential for conflicts)


In [13]:
# git-update-force test 
# should pull
git-update-force

Not zipping changes because PRESERVE_GIT_STASH not 1
issuing: git stash
issuing: git pull --all
issuing: git stash pop
Not unzipping changes (because PRESERVE_GIT_STASH not 1)
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   hello.txt

no changes added to commit (use "git add" and/or "git commit -a")
Dropped refs/stash@{0} (bb124162a8924f9a9f4f298f6eb1f18925ad9808)



In [14]:
#assert
[ "$(cat hello2.txt)" == "second change" ]; echo $?;

0


In [15]:
# undo changes and remove newly created files

# TODO2:
## also, converting unique ID's to codes like temp-git-test/.git/objects/pack/pack-HEX.idx
## EX: cat $log | perl -pe 's/\b[0-9]+\b/NUM/g; s/\b[a-f0-9]+\b/HEX/g;'

git restore .
( git rm -f *.txt && git add . ) > /dev/null
git commit -m "cleanup test files"
git push

[main ed884f0] cleanup test files
 2 files changed, 2 deletions(-)
 delete mode 100644 hello.txt
 delete mode 100644 hello2.txt
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 6 threads
Compressing objects: 100% (1/1), done.
Writing objects: 100% (2/2), 243 bytes | 243.00 KiB/s, done.
Total 2 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   4cf7bac..ed884f0  main -> main


In [16]:
# delete test local repos

command cd "/tmp"
rm -rf temp-git-test > /dev/null
rm -rf temp-git-test-B > /dev/null
